In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import pprint
import requests
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains 

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.nasa_db
collection = db.items

<b>Gathering News Articles</b>

In [66]:
executable_path = {'executable_path': 'C:\\Users\\calin\\Desktop\\HomeWork\\Module-12\\MissionToMars\\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [8]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

results = soup.find_all('div', class_='list_text')

for result in results:
    news_title = result.find('div', class_="content_title").text   
    news_body = result.find('div', class_="article_teaser_body").text   
    print(news_title)
    print(news_body)
    post = {
            'title': news_title,
            'Content': news_body
            }

    collection.insert_one(post)

NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.
NASA's Mars Helicopter Attached to Mars 2020 Rover 
The helicopter will be first aircraft to perform flight tests on another planet.
What's Mars Solar Conjunction, and Why Does It Matter?
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.
Scientists Explore Outback as Testbed for Mars 
Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars.
NASA-JPL Names 'Rolling Stones Rock' on Mars
NASA's Mars InSight mission honored one of the biggest bands of all time at Pasadena concert.
Robotic Toolkit Added to NASA's Mars 2020 Rover
The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's

<b>JPL Mars Images</b>

In [44]:
jpl_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_image)

html_image = browser.html

soup = BeautifulSoup(html_image, 'html.parser')
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
nasa_url = 'https://www.jpl.nasa.gov'

featured_image_url = nasa_url + featured_image_url

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg'

<b>Mars Weather</b>


In [48]:
weath_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weath_url)

mars_weather = browser.html
soup = BeautifulSoup(weather_url, 'html.parser')

tweets = soup.find_all('div', class_='js-tweet-text-container')
for tweet in tweets: 
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPapic.twitter.com/9YLawm67zS


<b>Mars Facts</b>

In [62]:
mars_url = "https://space-facts.com/mars/"
browser.visit(mars_url)

facts_html = browser.html
soup = BeautifulSoup(facts_html, 'html.parser')

mars_table = soup.find('table', class_="tablepress tablepress-id-p-mars")
mars_facts = mars_table.find_all('tr')

key = []
value = []

#Loop to append all table data into Key and Value lists
for row in mars_facts:
    table_data = row.find_all('td')
    key.append(table_data[0].text)
    value.append(table_data[1].text)
    


In [63]:
mars_df = pd.DataFrame({
    "Property": key,
    "Value": value
})

mars_df

,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [61]:
mars_facts_html = mars_df.to_html(header = False, index = False)
mars_facts_html

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

<b>Mars Hemisphers images</b>

In [67]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]